# Isotropic Scaling Experiment

## Synopsis

In this experiment, I will be looking at how the isotropic scaling effects the HSIC score for the HSIC and KA algorithms. In theory, because we are trying to find one parameter shared between the two kernel functions, there should be problems when the scale of one distribution is larger than another. It's a drawback of the method and it motivates the need to use two different parameters for the distributions.

## Code

In [1]:
import sys, os
# Insert path to model directory,.
cwd = os.getcwd()
path = f"{cwd}/../../src"
sys.path.insert(0, path)

import warnings
import tqdm
import random
import pandas as pd
import numpy as np
import argparse
from sklearn.utils import check_random_state

# toy datasets
from data.toy import generate_dependence_data, generate_isotropic_data

# Kernel Dependency measure
from models.train_models import get_gamma_init
from models.train_models import get_hsic
from models.kernel import estimate_sigma, sigma_to_gamma, gamma_to_sigma, get_param_grid
from models.ite_algorithms import run_rbig_models
from sklearn.preprocessing import StandardScaler

# Plotting
from visualization.distribution import plot_scorer
from visualization.scaling import plot_scorer_scale, plot_scorer_scale_norm


# experiment helpers
from tqdm import tqdm

# Plotting Procedures
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# plt.style.use(['fivethirtyeight', 'seaborn-poster'])
warnings.filterwarnings('ignore') # get rid of annoying warnings
%matplotlib inline


%load_ext autoreload
%autoreload 2

In [2]:
plt.style.available

['seaborn-dark-palette',
 'classic',
 'ggplot',
 'seaborn-dark',
 'seaborn-pastel',
 'seaborn-bright',
 'seaborn-deep',
 'tableau-colorblind10',
 'seaborn-talk',
 'fast',
 'seaborn-ticks',
 'seaborn-white',
 'bmh',
 'fivethirtyeight',
 'seaborn-muted',
 '_classic_test',
 'grayscale',
 'seaborn-darkgrid',
 'seaborn-poster',
 'seaborn',
 'seaborn-whitegrid',
 'dark_background',
 'seaborn-paper',
 'seaborn-colorblind',
 'seaborn-notebook',
 'Solarize_Light2']

### Experimental Design

The objective of this experiment is to measure how the Mutual information (MI) changes related to the HSIC score of different methods when we change the data and preprocessing conditions (normalization and scale). We change the nature of the data via the scale of the data received and whether or not we do a normalization procedure before we submit the datasets to our HSIC algorithms. Each HSIC method will give us a score and we can calculate the Mutual information

**Free Params**

* Number of Trials (`seed`)
    * 1:10
* Scale or not scaled (`scale`)
* Normalized | Not Normalized (`normalize`)
* HSIC Algorithm (`method`)
    * HSIC, KA, cKA
* Dataset (`dataset`)
    * Linear, Sinusoidal, Circle, Random
* Amount of Noise (`noise` List)
    * log space 

**Measurements**

* Mutual Information (`mi`)
* HSIC score (`score`)
* Time for execution (`time`)

**Fixed Parameters**

* Number of points (`num_points`)
* Noise for X points (`noise_x`)
* Noise for Y points (`noise_y`)
* 

## Demo

In [31]:
np.logspace(-2, 0, 10)

array([0.01      , 0.01668101, 0.02782559, 0.04641589, 0.07742637,
       0.12915497, 0.21544347, 0.35938137, 0.59948425, 1.        ])

In [3]:
class DataParams:
    dataset = 'line'
    num_points = 500
    noise_y = 0.1
    alpha = 1.0
    beta = 1.0

class ExpParams:
    dataset = ['line', 'sine', 'circ', 'rand']
    seed = np.linspace(1,10,10)
    scale = np.logspace(-2, 2, 10)
    normalized = [True, False]
    noise = np.logspace(-3, 1, 10)
    method = ['hsic', 'tka', 'ctka']
    gamma_method = [
        ('median', 0.2, None),
        ('median', 0.4, None),
        ('median', 0.5, None),
        ('median', 0.6, None),
        ('median', 0.8, None),
    ]

#### Helper Functions

In [4]:
from typing import Tuple, Type, Optional

def get_gamma_name(gamma_method: Tuple[str,str,str])-> str:
    if gamma_method[1] is None and gamma_method[2] is None:
        gamma_name = gamma_method[0]
    elif gamma_method[1] is not None and gamma_method[2] is None:
        gamma_name = f"{gamma_method[0]}_p{gamma_method[1]}"
    elif gamma_method[1] is None and gamma_method[2] is not None:
        gamma_name = f"{gamma_method[0]}_s{gamma_method[2]}"
    elif gamma_method[1] is not None and gamma_method[2] is not None:
        gamma_name = f"{gamma_method[0]}_s{gamma_method[1]}_s{gamma_method[2]}"
    else:
        raise ValueError('Unrecognized Combination...')
    return gamma_name

def plot_data(X: np.ndarray, Y: np.ndarray):
    fig, ax = plt.subplots(nrows=1, figsize=(7, 5))

    ax.scatter(X, Y, color='red')
    # plt.legend(fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()
    plt.show()

In [5]:
class ScaleExperiment:
    def __init__(self, data_params, exp_params):
        self.data_params = data_params
        self.exp_params = exp_params
        
    def _get_data(self, dataset: str, noise: float, seed: int)-> Tuple[np.ndarray, np.ndarray]:
        """Gathers the raw dependence data"""
        # get dataset
        X, Y = generate_dependence_data(
            dataset=dataset,
            num_points=self.data_params.num_points,
            seed=seed,
            noise_x=noise,
            noise_y=noise,
            alpha=self.data_params.alpha,
            beta=self.data_params.beta
        )
        return X, Y
    
    def _apply_noise(self, X: np.ndarray, Y: np.ndarray, noise: float, seed: int)-> Tuple[np.ndarray, np.ndarray]:
        
        rng = check_random_state(seed)
        
        X += rng.randn(X.shape[0], X.shape[1])
#         Y += rng.randn(Y.shape)
        
        
        return X, Y
    def _apply_scaling(self, X: np.ndarray, scale: float)-> np.ndarray:
        """The scaling step in our experiment"""
        # apply scaling
        return scale * X
    
    def _apply_normalization(self, X: np.ndarray, Y: np.ndarray, normalize: bool)-> np.ndarray:
        """The normalization step in our experiment."""
        # apply normalization
        if normalize == True:
            X = StandardScaler().fit_transform(X)
#             Y = StandardScaler().fit_transform(Y)
        elif normalize == False:
            pass
        else:
            raise ValueError(f'Unrecognized boolean value for normalize {normalize}')
        return X, Y
    
    def _apply_mi_estimate(self, X: np.ndarray, Y: np.ndarray)-> float:
        """Apply Mutual Information estimator. 
        We choose to use RBIG as our estimator."""
        # estimate mutual information
        mi, _ = run_rbig_models(X, Y, measure='mi', verbose=None)
        
        return mi
    
    def _apply_hsic_estimate(
        self, 
        X: np.ndarray, 
        Y: np.ndarray, 
        method: str, 
        gamma_init: Tuple[str, Optional[float], Optional[float]])-> float:
        """Apply HSIC estimator using one of the 3 algorithms:
        * HSIC
        * KA
        * cKA
        """
        # initialize the gamma parameter
        gamma_init = get_gamma_init(X, Y, gamma_init[0], gamma_init[1], gamma_init[2])
        
        # get hsic_value
        hsic_value = get_hsic(X, Y, method, gamma_init, maximum=False)
        
        return hsic_value
    
    def _experiment_step(
        self,
        results_df: pd.DataFrame,
        dataset: str,
        noise: float, seed: int,
        scale: float,
        normalize: bool,
        method: str,
        gamma_init: Tuple[str, Optional[float], Optional[float]]
    )-> pd.DataFrame:
        
        # Step I - Extract Data
        X, Y = self._get_data(dataset=dataset, noise=noise, seed=seed)
        
#         # Step I.1 - Apply Noise
#         X, Y = self._apply_noise(X=X, Y=Y, noise=noise, seed=seed)
        
        # Step II - Apply Scaling
        X = self._apply_scaling(X=X, scale=scale)

        # Step III - Apply Normalization
        X, Y = self._apply_normalization(X=X, Y=Y, normalize=normalize)

        # Step IV - Estimate mutual information
        mi = self._apply_mi_estimate(X, Y)

        # Step IV - Estimate HSIC value
        hsic_value = self._apply_hsic_estimate(X, Y, method, gamma_init)
        
        # Step V - Save Results to dataframe
        results_df = results_df.append({
            'normalized': normalize,
            'trial': seed,
            'dataset': dataset,
            'scale': scale,
            'scorer': method,
            'gamma_method': get_gamma_name(gamma_init),
            'hsic_value': hsic_value,
            "mi": mi,
            "noise": noise,
        }, ignore_index=True)
        return results_df
    
    def run_experiment(self):
        
        
        results_df = pd.DataFrame()
#         print(self.exp_params.seed)
        
        # Loop Through Free Parameters
        for iseed in self.exp_params.seed:
#             print(iseed)
            for idataset in self.exp_params.dataset:
                for inoise in self.exp_params.noise: 
                    for iscale in self.exp_params.scale:
                        for inormalize in self.exp_params.normalized:
                            for igamma in self.exp_params.gamma_method:
                                for imethod in self.exp_params.method:
                                    results_df = self._experiment_step(
                                        results_df=results_df,
                                        dataset=idataset,
                                        noise=inoise, 
                                        seed=iseed,
                                        scale=iscale,
                                        normalize=inormalize,
                                        method=imethod,
                                        gamma_init=igamma
                                    )
        return results_df
            

#### Test Run - Full Algorithm

In [6]:
# Initialize Experiment class
exp_class = ScaleExperiment(DataParams, ExpParams, )

# ========================================
# Step I - Extract data
# ========================================
dataset = 'circ'
noise = 0.000
seed = 1

X, Y = exp_class._get_data(dataset=dataset, noise=noise, seed=seed)

# plot_data(X,Y)
# ========================================
# Step II - Apply Scaling
# ========================================
scale = 1.

X = exp_class._apply_scaling(X=X, scale=scale)
# plot_data(X,Y)

# ========================================
# Step III - Apply Normalization
# ========================================
normalize = False

X, Y = exp_class._apply_normalization(X=X, Y=Y, normalize=normalize)

# plot_data(X,Y)

# ========================================
# Step IV - Estimate mutual information
# ========================================
mi = exp_class._apply_mi_estimate(X, Y)

print(f'MI (RBIG): {mi:.4f}')

# ========================================
# Step V - Estimate HSIC value
# ========================================
method = 'ctka'
gamma_init = ('median', 0.5, None)

hsic_value = exp_class._apply_hsic_estimate(X, Y, method, gamma_init)

print(f'HSIC score ({method}): {hsic_value:.4f}')

MI (RBIG): 1.8373
HSIC score (ctka): 0.0743


#### Test Run - Experimental Step

In [127]:
# Initialize Experiment class
exp_class = ScaleExperiment(DataParams, ExpParams, )


results_df = exp_class._experiment_step(
    dataset=dataset, noise=noise, seed=seed,
    scale=scale,
    normalize=normalize,
    method=method,
    gamma_init=gamma_init
)

In [128]:
results_df.head()

,dataset,gamma_method,hsic_value,mi,normalized,scale,scorer,trial
0,line,median_p0.5,0.713516,1.075195,1.0,10.0,ctka,123.0


#### Test Run - Full Experiment Loop

In [201]:
class DataParams:
    dataset = 'line'
    num_points = 1_000
    noise_y = 0.00
    alpha = 1.0
    beta = 1.0

class ExpParams:
    dataset = ['line', 'sine', 'circ', 'rand']
    seed = np.linspace(1,10,10, dtype=int)
    scale = np.logspace(-2, 2, 10)
    normalized = [True, False]
    noise = [0.01]
    method = ['hsic', 'tka', 'ctka']
    gamma_method = [
#         ('median', 0.2, None),
#         ('median', 0.4, None),
        ('median', 0.5, None),
#         ('median', 0.6, None),
#         ('median', 0.8, None),
    ]

# Initialize Experiment class
exp_class = ScaleExperiment(DataParams, ExpParams, )

results_df = exp_class.run_experiment()

1


KeyboardInterrupt: 

In [180]:
results_df.tail()

,dataset,gamma_method,hsic_value,mi,normalized,scale,scorer,trial
11995,rand,median_p0.6,7.162945e-01,0.02152,0.0,100.0,tka,1.0
11996,rand,median_p0.6,4.157931e-03,0.02152,0.0,100.0,ctka,1.0
11997,rand,median_p0.8,1.453113e-07,0.02152,0.0,100.0,hsic,1.0
11998,rand,median_p0.8,8.396995e-01,0.02152,0.0,100.0,tka,1.0
11999,rand,median_p0.8,4.510840e-03,0.02152,0.0,100.0,ctka,1.0


### Visualization

In [ ]:
def plot_results(
    df: pd.DataFrame,
    case: int,
    dataset: str='line'
    scorer: str='hsic'
):
    df = df[df['dataset'] == dataset]
    df = df[df['scorer'] == scorer]
    
    # Case I - Unscaled, Unnormalized
    if case == 1:
        
        df = df[df['normalized'] == 0.0]
        

#### Case I - Unscaled, Unnormalized

In [ ]:
sub_df = results_df[results_df['dataset'] == 'sine']
sub_df = sub_df[sub_df['normalized'] == 0.0]
# sub_df = sub_df[sub_df['gamma_method'] == 'median_p0.6']
sub_df = sub_df[sub_df['scorer'] == 'ctka']
sub_df = sub_df[sub_df['trial'] == 1.0]
# sub_df['mi'] = np.log2(sub_df['mi'])

# plot data
fig, ax = plt.subplots()

pts = sns.scatterplot(
    x='hsic_value',
    y='mi',
    data=sub_df
)
plt.show()

In [162]:
sub_df.head()

,dataset,gamma_method,hsic_value,mi,normalized,scale,scorer,trial
2,line,median_p0.2,0.575708,0.969319,1.0,0.01,ctka,1.0
5,line,median_p0.4,0.700491,0.969319,1.0,0.01,ctka,1.0
8,line,median_p0.5,0.735901,0.969319,1.0,0.01,ctka,1.0
11,line,median_p0.6,0.780966,0.969319,1.0,0.01,ctka,1.0
14,line,median_p0.8,0.847334,0.969319,1.0,0.01,ctka,1.0


## Cases - Walkthrough

In [21]:
def get_params(case: int):

    # Case I - Unscaled, Unnormalized
    if case == 1:

        class DataParams:
            num_points = 2_000
            noise_y = 0.0
            alpha = 1.0
            beta = 1.0

        class ExpParams:
            dataset = [
                "line",
                "sine",
                "circ",
                # "rand"
            ]
            seed = np.linspace(1, 10, 10, dtype=int)
            scale = [1.0]
            normalized = [False]
            noise = np.logspace(-2, 1, 10)
            method = ["hsic", "tka", "ctka"]
            gamma_method = [
                ("median", 0.2, None),
                ("median", 0.4, None),
                ("median", 0.5, None),
                ("median", 0.6, None),
                ("median", 0.8, None),
            ]

    # Case II - Unscaled, Normalized
    elif case == 2:

        class DataParams:
            num_points = 2_000
            noise_y = 0.0
            alpha = 1.0
            beta = 1.0

        class ExpParams:
            dataset = [
                "line",
                "sine",
                "circ",
                # "rand"
            ]
            seed = np.linspace(1, 10, 10, dtype=int)
            scale = [1.0]
            normalized = [True]
            noise = np.logspace(-2, 1, 10)
            method = ["hsic", "tka", "ctka"]
            gamma_method = [
                ("median", 0.2, None),
                ("median", 0.4, None),
                ("median", 0.5, None),
                ("median", 0.6, None),
                ("median", 0.8, None),
            ]

    # Case III - Scaled, Unnormalized
    elif case == 3:

        class DataParams:
            num_points = 2_000
            noise_y = 0.01
            alpha = 1.0
            beta = 1.0

        class ExpParams:
            dataset = [
                "line",
                "sine",
                "circ",
                # "rand",
            ]
            seed = np.linspace(1, 10, 10, dtype=int)
            scale = np.logspace(-2, 2, 10)
            normalized = [False]
            noise = np.logspace(-2, 1, 10)
            method = ["hsic", "tka", "ctka"]
            gamma_method = [
                ("median", 0.2, None),
                ("median", 0.4, None),
                ("median", 0.5, None),
                ("median", 0.6, None),
                ("median", 0.8, None),
            ]

    elif case == 4:

        class DataParams:
            dataset = "line"
            num_points = 2_000
            noise_y = 0.01
            alpha = 1.0
            beta = 1.0

        class ExpParams:
            dataset = [
                "line",
                "sine",
                "circ",
                # "rand"
            ]
            seed = np.linspace(1, 10, 10, dtype=int)
            scale = np.logspace(-2, 2, 10)  # [0.01, 1.0, 100.0]
            normalized = [True]
            noise = np.logspace(-3, 1, 10)
            method = ["hsic", "tka", "ctka"]
            gamma_method = [
                ("median", 0.2, None),
                ("median", 0.4, None),
                ("median", 0.5, None),
                ("median", 0.6, None),
                ("median", 0.8, None),
            ]

    else:
        raise ValueError(f"Unrecognized case: '{case}'")

    return DataParams, ExpParams

#### Case I - Unscaled, Unnormalized

For this first walkthrough, we are assuming that the data is unscaled and that the data is unnormalized. 

**Hypothesis**: We all methods should showcase some relationship to the amount of Mutual information but it will not necessarily be a strict relationship. Thinking from the previous results, the KA method should perform the worst, the HSIC method should perform OK with some inconsistencies and the CKA should perform the best and showcase a trend.

In [ ]:
# case number
case = 1

# get parameters
DataParams, ExpParams = get_params(case=case)

# Initialize Experiment class
exp_class = ScaleExperiment(DataParams, ExpParams, )

results_df = exp_class.run_experiment()

In [94]:
results_df

,dataset,gamma_method,hsic_value,mi,noise,normalized,scale,scorer,trial
0,line,median_p0.5,0.110588,3.094753,0.01,0.0,1.0,hsic,1.0
1,line,median_p0.5,0.999366,3.094753,0.01,0.0,1.0,tka,1.0
2,line,median_p0.5,0.998172,3.094753,0.01,0.0,1.0,ctka,1.0
3,line,median_p0.5,0.105917,3.074389,0.01,0.0,1.0,hsic,2.0
4,line,median_p0.5,0.999368,3.074389,0.01,0.0,1.0,tka,2.0
5,line,median_p0.5,0.998062,3.074389,0.01,0.0,1.0,ctka,2.0
6,line,median_p0.5,0.110391,3.111298,0.01,0.0,1.0,hsic,3.0
7,line,median_p0.5,0.999400,3.111298,0.01,0.0,1.0,tka,3.0
8,line,median_p0.5,0.998244,3.111298,0.01,0.0,1.0,ctka,3.0


#### Case II - Unscaled, Normalized

In this case, we see that

In [102]:
# case number
case = 2

# get parameters
DataParams, ExpParams = get_params(case=case)

# Initialize Experiment class
exp_class = ScaleExperiment(DataParams, ExpParams, )

# Run experiment
results_df = exp_class.run_experiment()

In [103]:
results_df

,dataset,gamma_method,hsic_value,mi,noise,normalized,scale,scorer,trial
0,line,median_p0.5,0.110589,3.094753,0.01,1.0,1.0,hsic,1.0
1,line,median_p0.5,0.999366,3.094753,0.01,1.0,1.0,tka,1.0
2,line,median_p0.5,0.998172,3.094753,0.01,1.0,1.0,ctka,1.0
3,line,median_p0.5,0.089667,1.011999,0.10,1.0,1.0,hsic,1.0
4,line,median_p0.5,0.946835,1.011999,0.10,1.0,1.0,tka,1.0
5,line,median_p0.5,0.843181,1.011999,0.10,1.0,1.0,ctka,1.0
6,line,median_p0.5,0.003926,0.043903,1.00,1.0,1.0,hsic,1.0
7,line,median_p0.5,0.686470,0.043903,1.00,1.0,1.0,tka,1.0
8,line,median_p0.5,0.040370,0.043903,1.00,1.0,1.0,ctka,1.0


#### Case III - Scaled, Unormalized

In [108]:
# case number
case = 3

# get parameters
DataParams, ExpParams = get_params(case=case)

# Initialize Experiment class
exp_class = ScaleExperiment(DataParams, ExpParams, )

# Run Experiment
results_df = exp_class.run_experiment()

In [109]:
results_df

,dataset,gamma_method,hsic_value,mi,noise,normalized,scale,scorer,trial
0,line,median_p0.5,1.109297e-04,3.094753,0.01,0.0,0.01,hsic,1.0
1,line,median_p0.5,6.316801e-01,3.094753,0.01,0.0,0.01,tka,1.0
2,line,median_p0.5,6.369142e-01,3.094753,0.01,0.0,0.01,ctka,1.0
3,line,median_p0.5,1.105885e-01,3.094753,0.01,0.0,1.00,hsic,1.0
4,line,median_p0.5,9.993656e-01,3.094753,0.01,0.0,1.00,tka,1.0
5,line,median_p0.5,9.981718e-01,3.094753,0.01,0.0,1.00,ctka,1.0
6,line,median_p0.5,1.099729e-04,3.094753,0.01,0.0,100.00,hsic,1.0
7,line,median_p0.5,6.323451e-01,3.094753,0.01,0.0,100.00,tka,1.0
8,line,median_p0.5,6.371835e-01,3.094753,0.01,0.0,100.00,ctka,1.0
9,line,median_p0.5,8.439792e-05,0.963049,0.10,0.0,0.01,hsic,1.0


#### Case IV - Scaled, Normalized

In [106]:
# case number
case = 4

# get parameters
DataParams, ExpParams = get_params(case=case)

# Initialize Experiment class
exp_class = ScaleExperiment(DataParams, ExpParams, )

# Run Experiment
results_df = exp_class.run_experiment()

In [107]:
results_df

,dataset,gamma_method,hsic_value,mi,noise,normalized,scale,scorer,trial
0,line,median_p0.5,0.110589,3.094753,0.01,1.0,0.01,hsic,1.0
1,line,median_p0.5,0.999366,3.094753,0.01,1.0,0.01,tka,1.0
2,line,median_p0.5,0.998172,3.094753,0.01,1.0,0.01,ctka,1.0
3,line,median_p0.5,0.110589,3.094753,0.01,1.0,1.00,hsic,1.0
4,line,median_p0.5,0.999366,3.094753,0.01,1.0,1.00,tka,1.0
5,line,median_p0.5,0.998172,3.094753,0.01,1.0,1.00,ctka,1.0
6,line,median_p0.5,0.110589,3.094753,0.01,1.0,100.00,hsic,1.0
7,line,median_p0.5,0.999366,3.094753,0.01,1.0,100.00,tka,1.0
8,line,median_p0.5,0.998172,3.094753,0.01,1.0,100.00,ctka,1.0
9,line,median_p0.5,0.089667,1.036774,0.10,1.0,0.01,hsic,1.0
